In [2]:
import numpy as np
from time import time
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.preprocessing import scale



In [4]:
X_digits, y_digits = load_digits(return_X_y=True)
data = scale(X_digits)
 
n_samples, n_features = data.shape
n_digits = len(np.unique(y_digits))
labels = y_digits
 
sample_size = 1000
 
print("n_digits: %d, \t n_samples %d, \t n_features %d"
      % (n_digits, n_samples, n_features))

 


n_digits: 10, 	 n_samples 1797, 	 n_features 64


In [5]:
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette')

init		time	inertia	homo	compl	v-meas	ARI	AMI	silhouette


In [6]:
 
def bench_k_means(estimator, name, data):
    t0 = time()
    estimator.fit(data)
    print('%-9s\t%.2fs\t%i\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f'
          % (name, (time() - t0), estimator.inertia_,
             metrics.homogeneity_score(labels, estimator.labels_),
             metrics.completeness_score(labels, estimator.labels_),
             metrics.v_measure_score(labels, estimator.labels_),
             metrics.adjusted_rand_score(labels, estimator.labels_),
             metrics.adjusted_mutual_info_score(labels,  estimator.labels_),
             metrics.silhouette_score(data, estimator.labels_,
                                      metric='euclidean',
                                      sample_size=sample_size)))

In [7]:
bench_k_means(KMeans(init='k-means++', n_clusters=n_digits, n_init=100),
              name="k-means++", data=data)
 
bench_k_means(KMeans(init='random', n_clusters=n_digits, n_init=100),
              name="random", data=data)

k-means++	1.34s	69405	0.603	0.651	0.626	0.467	0.622	0.149
random   	0.81s	69409	0.605	0.653	0.628	0.468	0.625	0.152


In [10]:
!pip install diffprivlib 
from diffprivlib.models import KMeans
 


In [11]:
bench_k_means(KMeans(epsilon=1.0, bounds=None, n_clusters=n_digits, init='k-means++', n_init=100), name="dp_k-means", data=data)

c:\python39\lib\site-packages\diffprivlib\utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'init' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "
c:\python39\lib\site-packages\diffprivlib\utils.py:71: DiffprivlibCompatibilityWarning: Parameter 'n_init' is not functional in diffprivlib.  Remove this parameter to suppress this warning.
  warnings.warn(f"Parameter '{arg}' is not functional in diffprivlib.  Remove this parameter to suppress this "
c:\python39\lib\site-packages\diffprivlib\models\k_means.py:138: PrivacyLeakWarning: Bounds have not been specified and will be calculated on the data provided.  This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify `bounds` for each dimension.
  warnings.warn("Bounds have not been specified and will be calculated on the data pr

dp_k-means	0.41s	311778	0.001	0.271	0.001	-0.000	-0.000	0.788
